In [1]:
import numpy as np
from PIL import Image
from sklearn.preprocessing import OneHotEncoder

In [2]:
N_CLASS = 40
N_SAMPLES = 10
TOTAL_SAMPLES = 400
TEST_PERCENTAGE = 0.2
TEST_SAMPLES = int(TOTAL_SAMPLES * TEST_PERCENTAGE)
TRAIN_SAMPLES = int(TOTAL_SAMPLES - TEST_SAMPLES)
COLLECTION_FILE_NAME = 'data' 

In [3]:
import multiprocessing as mp

def read_data(f_index):    
    train_X = []
    train_Y = []
    test_X = []
    test_Y = []
    for img_num in range(N_SAMPLES):
        image = np.array(Image.open(
            f"orl_faces/s{f_index + 1}/{img_num + 1}.pgm").getdata())
        if (img_num in range(N_SAMPLES)[-int(N_SAMPLES*TEST_PERCENTAGE):]):
            test_X.append(image)
            test_Y.append(f_index)
        else:
            train_X.append(image)
            train_Y.append(f_index)
            
    return np.asarray(train_X), np.asarray(train_Y), np.asarray(test_X), np.asarray(test_Y)

In [4]:
%%time
processes_args = ((i) for i in range(N_CLASS))
with mp.Pool(4) as p:
    results = p.map(read_data, processes_args)
    
X_train = list(x[0] for x in results)
X_train = np.concatenate(X_train)
Y_train = list(x[1] for x in results)
Y_train = np.concatenate(Y_train)

X_test = list(x[2] for x in results)
X_test = np.concatenate(X_test)
Y_test = list(x[3] for x in results)
Y_test = np.concatenate(Y_test)

print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
np.save(f"train_{COLLECTION_FILE_NAME}_x", X_train)
np.save(f"train_{COLLECTION_FILE_NAME}_y", Y_train)
np.save(f"test_{COLLECTION_FILE_NAME}_x", X_test)
np.save(f"test_{COLLECTION_FILE_NAME}_y", Y_test)

(320, 10304) (320,) (80, 10304) (80,)
CPU times: user 46.9 ms, sys: 219 ms, total: 266 ms
Wall time: 587 ms


In [5]:
# Load dataset function

def load_dataset():
    train_X = np.load(f"train_{COLLECTION_FILE_NAME}_x.npy")
    train_Y = np.load(f"train_{COLLECTION_FILE_NAME}_y.npy")
    test_X = np.load(f"test_{COLLECTION_FILE_NAME}_x.npy")
    test_Y = np.load(f"test_{COLLECTION_FILE_NAME}_y.npy")
    return train_X, train_Y, test_X, test_Y
    
X_train, Y_train, X_test, Y_test = load_dataset() 
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

(320, 10304) (320,) (80, 10304) (80,)


In [7]:
# Scale images

X_train = X_train/255.
X_test  = X_test/255.

In [8]:
# Function for encode categorical labels

def encode_labels(df_train, df_test):
    df = np.concatenate((df_train, df_test))
    enc = OneHotEncoder(sparse=False, handle_unknown='ignore')
    df = enc.fit_transform((df).reshape((-1, 1)))
    return df[:-80], df[320:]

Y_train_encoded, Y_test_encoded = encode_labels(Y_train, Y_test)
print(Y_train_encoded.shape, Y_test_encoded.shape)

(320, 40) (80, 40)


In [9]:
# Implementation of NN with helper functions

def sigmoid(z):
    return 1/(1 + np.exp(-z))

def initialize_with_zeros(dim):
    w = np.zeros((dim, 40))
    b = 0

    assert(w.shape == (dim, 40))
    assert(isinstance(b, float) or isinstance(b, int))
    
    return w, b

def propagate(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) + b)                           # compute activation
    cost = - np.mean(Y * np.log(A) + (1 - Y) * np.log(1 - A)) # compute cost
    dw = np.dot(X, (A - Y).T) / m
    db = np.mean(A - Y)
    
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost = np.squeeze(cost)
    assert(cost.shape == ())
    
    grads = {"dw": dw,
             "db": db}
    
    return grads, cost

def optimize(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)

        dw = grads["dw"]
        db = grads["db"]
        
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        if i % 100 == 0:
            costs.append(cost)

        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    params = {"w": w,
              "b": b}
    
    grads = {"dw": dw,
             "db": db}
    
    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]
    
    w = w.reshape(X.shape[0], 40)
    
    A = sigmoid(np.dot(w.T, X) + b)
    Y_prediction = np.round(A)
        
    assert(Y_prediction.shape == (40, m))
    
    return Y_prediction

def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    w, b = initialize_with_zeros(X_train.shape[0])

    parameters, grads, costs = optimize(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    
    w = parameters["w"]
    b = parameters["b"]
    
    Y_prediction_test = predict(w, b, X_test)
    Y_prediction_train = predict(w, b, X_train)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))

    d = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations}
    
    return d

In [10]:
d = model(X_train.T, Y_train_encoded.T, X_test.T, Y_test_encoded.T, num_iterations = 2000, learning_rate = 0.005, print_cost = True)

SyntaxError: invalid syntax (<ipython-input-10-ee4f4e1c4c44>, line 1)